# "Data Mining Final Project"

> "Data Mining Final Project"

- toc:true
- branch: master
- badges: true
- image: /images
- comments: true
- author: Hyeokju Yeon
- categories: [project, data mining, excersize]

### 부동산 공급과 수요에 따른 집값 분석 


**목차**
* [1. 서론](#chapter1)
    * [1-1. 주제 선정이유](#section_1_1)
* [2. 데이터 소개](#chapter2)
    * [2-1. 데이터 제공 대회](#section_2_1)
    * [2-2. 데이터 소개](#section_2_2)
* [3. 분석](#chapter3)
    * [3-1. 네이버 뉴스 제목 워드클라우드](#section_3_1)
    * [3-2. 전국 규모 아파트 가격](#section_3_2)
    * [3-3. 시군구 규모 아파트 가격](#section_3_3)
    * [3-4. 수요과 공급](#section_3_4)
    * [3-5. 타깃 설정과 수요와 공급 적용](#section_3_5)
* [4. 결론](#chapter4)
    * [4-1. 한계점](#section_4_1) 
    * [4-2. 추후 연구 방향](#section_4_2) 

### 1. 서론 <a class="anchor" id="chapter1"></a>

- **주제** : 부동산 공급과 수요에 따른 집값 분석 -> 집이 부족하여 집값이 계속 상승하는 것인가?<br></br>

- Load Packages

In [ ]:
import pandas as pd
import geopandas as gpd 
import plotly.express as px 
import json
from bs4 import BeautifulSoup
import requests
from konlpy.tag import Twitter
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

#### 1-1. 주제 선정이유 <a class="anchor" id="section_1_1"></a>

- **뉴스에서도 항상 나오는 꾸준히 상승하는 집값에 대해 집이 부족하여 집값이 상승하는 것인가 궁금증을 해소하기 위함이다.**

---

### 2. 데이터 소개 <a class="anchor" id="chapter2"></a>

#### 2-1. 데이터 제공 대회 <a class="anchor" id="section_2_1"></a>
- [**데이콘**](https://dacon.io/)


#### 2-2. 데이터 소개 <a class="anchor" id="section_2_2"></a>
- **['한국 부동산 데이터 시각화 경진대회'](https://dacon.io/competitions/official/235724/overview/description)에서 제공한 데이터**
    - 주택매매가격지수: 주택매매가격의 증감폭을 년월 별로 나타낸 데이터
    - 주택전세가격지수: 주택전세가격의 증감폭을 년월 별로 나타낸 데이터
    - 미분양주택현황: 년월 별로 시도별 미분양주택의 세대수를 나타낸 데이터
    - 아파트 실거래가격지수: 주택 실거래가격의 증감폭을 년월 별로 나타낸 데이터
    - cf. 지수데이터란? 특정 년도 기준 증감 폭을 퍼센트로 나타낸 데이터(예시: 2017년도 기준 | 2017년도 데이터 100%, 2017년도 보다 증가 120%, 감소 80%) <p></p>
- **통계청 데이터 포털** 
    - [시군구 년도별 인구수 API](https://sgis.kostat.go.kr/developer/html/newOpenApi/api/dataApi/census.html): 년도별 시군구 인구수를 나타낸 데이터 <p></p>

- **시군구 지리정보**
    - [시군구 지리정보 데이터](http://www.gisdeveloper.co.kr/?p=2332): 시군구 지리정보를 담고 있는 데이터

In [ ]:
###### LOAD DATA

### DACON DATA
unsold_home = pd.read_csv("./data/미분양주택현황.csv")
Apartment_cost = pd.read_csv("./data/아파트 실거래가격지수.csv")
house_cost = pd.read_csv("./data/주택매매가격지수(KB).csv")
house_charter_cost = pd.read_csv("./data/주택전세가격지수(KB).csv")

### 시군구 지리정보
regions = gpd.read_file("./data/ctp_rvn.shp", encoding="cp949")

### 시군구 년도별 인구수 데이터 
population = pd.DataFrame()
for i in [2016,2017, 2018, 2019, 2020]:
    population_subset = pd.read_csv(f"{i}인구수.csv")
    population_subset["year"] = i
    population = pd.concat([population, population_subset])
    
population["house_cnt"] = population["house_cnt"]
population = population.reset_index(drop=True)

---

### 3. 분석 <a class="anchor" id="chapter3"></a>

#### 3-1. 네이버 뉴스 제목 워드클라우드 <a class="anchor" id="section_3_1"></a>

- 해당 코드는 원하는 키워드로 최신 네이버 뉴스의 제목을 크롤링해서 워드클라우드를 만들어 주는 코드이다.<p>
- 아래 집값에 관한 워드클라우드를 살펴보면, 상승과 하락이 빈도가 높은 것을 확인할 수 있다. 집값에 관해 나오는 기사는 하락과 상승에 대한 것이 많고, 많은 사람들 또한 관심이 있기 때문이라고 짐작할 수 있다.

In [ ]:
search_word = "집값"  # 검색어 지정
title_list = []
 
def get_titles(start_num, end_num):
    #start_num ~ end_num까지 크롤링
    while 1:
        if start_num > end_num:
            break
 
        url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}&start={}'.format(search_word,start_num)
        req = requests.get(url)
 
        # 정상적인 request 확인
        if req.ok:
            html = req.text
            soup = BeautifulSoup(html, 'html.parser')
 
            # 뉴스제목 뽑아오기
            titles = soup.select(
                'ul.list_news > li > div > div > a'
            )
 
            # list에 넣어준다
            for title in titles:
                title_list.append(title['title'])
        start_num += 10
 
def make_wordcloud(word_count):
    twitter = Twitter()
 
    sentences_tag = []
    #형태소 분석하여 리스트에 넣기
    for sentence in title_list:
        morph = twitter.pos(sentence)
        sentences_tag.append(morph)
 
    noun_adj_list = []
    #명사와 형용사만 구분하여 이스트에 넣기
    for sentence1 in sentences_tag:
        for word, tag in sentence1:
            if tag in ['Noun', 'Adjective']:
                noun_adj_list.append(word)
 
    #형태소별 count
    counts = Counter(noun_adj_list)
    tags = counts.most_common(word_count)
    
    #wordCloud생성
    #한글꺠지는 문제 해결하기위해 font_path 지정
    wc = WordCloud(font_path='C:\\Users\\user\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumGothicLight.ttf', background_color='white', width=800, height=600)
    cloud = wc.generate_from_frequencies(dict(tags))
    plt.figure(figsize=(10, 8))
    plt.axis('off')
    plt.imshow(cloud)
    plt.show()

if __name__ == '__main__':
    #1~200번게시글 까지 크롤링
    get_titles(1,100)
 
    #단어 30개까지 wordcloud로 출력
    make_wordcloud(30)

![wordcloud](./images/wordcloud.png)

#### 3-2. 전국 규모 아파트 가격 <a class="anchor" id="section_3_2"></a>
- 부동산 중 아파트를 타겟으로 전국의 아파트 실거래 지수를 살펴보았다.
- 해당 데이터는 2017년도를 기준으로 지수가 만들어졌기 때문에 2017년 100%인 1로 나타나 있는 것을 볼 수 있다. 
- 또한 전국적으로 아파트 실거래 지수는 계속해서 상승하고 있는것을 확인할 수 있다.<p>
- 전국적으로 상승하고 있지만, 미시적으로 시군구도 같은지 살펴보자

In [ ]:
### 주택 가격 지수 전처리 (2017-11-01이 기준이기 때문에 모든 데이터를 01-01로 한다)
def Apartment_cost_prepared(Apartment_cost):
    col1 = ["date"]
    col2 = [i.split("[")[0] for i in Apartment_cost.columns[1:].tolist()]
    col = col1 + col2
    Apartment_cost.columns = col
    Apartment_cost = Apartment_cost.fillna(0)
    
    year =  [f"{i}-11-01" for i in range(2006, 2022)]
    Apartment_cost = Apartment_cost.loc[Apartment_cost['date'].isin(year)]
    Apartment_cost["year"] = [f"{i}" for i in range(2006, 2021)]
    return Apartment_cost

Apartment_cost = Apartment_cost_prepared(Apartment_cost)

In [ ]:
Apartment_cost_total  = px.line(Apartment_cost, x="year", y="전국")
Apartment_cost_total.update_layout(
                                yaxis = dict(
                                    tickmode = 'array',
                                    tickvals = [i for i in range(60, 121, 10)],
                                    ticktext = [i/100 for i in range(60, 121, 10)]),
                                margin=dict(l=20, r=30, t=0, b=0),
                                template="plotly_dark")
Apartment_cost_total

![1](./images/1.png)

#### 3-3. 시군구 규모 아파트 가격 <a class="anchor" id="section_3_3"></a>

- 아래의 그래프로 보면 몇몇 시군구를 제외한 모든 지역이 장기적으로 지속해서 상승하고 있는것을 볼 수 있다.<p></p>
- 하지만 아래 그래프로 정확하게 살펴보기에는 한계가 있어보이니 지도로 나타내보자!

In [ ]:
region_cols = ['부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북', '충남',
 '전북', '전남', '경북', '경남', '제주', '서울']

Apartment_cost_each = px.line(Apartment_cost, x="year", y=region_cols)
Apartment_cost_each.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [i for i in range(2007, 2022)],
        ticktext = [f"{i}년" for i in range(2007, 2022)]),
    yaxis = dict(
        tickmode = 'array',
        tickvals = [i for i in range(30, 181, 30)],
        ticktext = [i/100 for i in range(30, 181, 30)]),
    margin=dict(l=0, r=0, t=0, b=0),
    template="plotly_dark"
)

Apartment_cost_each

![2](./images/2.png)

- 아래 지도 그림은 첫 번째는 기준 년도인 2017년 지도와 2020년 지도이다.<p></p>
- 2020년 지도를 보았을때 전체적으로 특별시,광역시 기준으로 실거래 지수가 증가한 것을 볼 수 있고, 그외 지역들은 소폭 감소한 것을 볼 수 있다.<p></p>

- 많은 지역을 자세히 살펴보면 좋겠지만, 타깃 지역을 설정하여 분석 추가적으로 살펴보겠다.
    - 2017년~2020년 아파트 가격이 가장 많이 떨어진 곳 : 경남 
    - 2017년~2020년 아파트 가격이 가장 많이 오른 곳 : 세종<p></p>

- 이 두곳을 중점적으로 살펴보겠다.

In [ ]:
Apartment_cost = Apartment_cost[["year", "서울", "부산", "대구", "인천", "광주", "대전", "울산", "세종", "경기", "강원", "충북", "충남", "전북", "전남", "경북", "경남", "제주"]]
Apartment_cost = pd.melt(Apartment_cost, id_vars=['year'], value_vars=[ "서울", "부산", "대구", "인천", "광주", "대전", "울산", "세종", "경기", "강원", "충북", "충남", "전북", "전남", "경북", "경남", "제주"]).sort_values('year')
Apartment_cost.columns = ["year", "region", "value"]


regions["CTP_KOR_NM"] = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주']
regions_geometry = regions[["CTP_KOR_NM", "geometry"]] 
regions_geometry.columns = ["region", "geometry"]

regions_geometry = regions_geometry.to_crs(4326)
regions_geometry = json.loads(regions_geometry.to_json())

Apartment_cost_2017 = Apartment_cost.loc[Apartment_cost["year"] == "2017"]
Apartment_cost_2020 = Apartment_cost.loc[Apartment_cost["year"] == "2020"]

In [ ]:
fig = px.choropleth_mapbox(Apartment_cost_2017, geojson=regions_geometry,
                     locations='region',
                     color='value',
                     color_continuous_scale='RdBu',
                     range_color=(40,170),
                     featureidkey="properties.region",
                     mapbox_style='carto-positron',
                     center = {"lat": 36.8467, "lon": 127.9462}, 
                     width=600,
                     zoom=6)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

![3](./images/3.png)

In [ ]:
fig = px.choropleth_mapbox(Apartment_cost_2020, geojson=regions_geometry,
                     locations='region',
                     color='value',
                     color_continuous_scale='RdBu',
                     range_color=(40,170),
                     featureidkey="properties.region",
                     mapbox_style='carto-positron',
                     center = {"lat": 36.8467, "lon": 127.9462}, 
                     width=600,
                     zoom=6)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

![4](./images/4.png)

#### 3-4. 수요와 공급 <a class="anchor" id="section_3_4"></a>

- 주제와 부합하여 타깃지역에 기본적인 경제 원리인 수요와 공급의 법칙이 적용되는지 확인보겠다. 
- 여기서 *수요*는 **인구 유입(증가)**, *공급*은 **미분양주택수**로 볼 예정이다.

#### 3-5. 타깃 설정과 수요와 공급 적용 <a class="anchor" id="section_3_5"></a>

- 타깃 지역은 위에서 언급한 대로 경남과 세종으로 하였다.<p></p>
- 경남과 세종의 미분양 주택의 양과 인구 증가율을 살펴보았을 때
    - 경남은 4개년 동안 꾸준하게 미분양된 주택수가 넘쳐났고, 인구 유입(증가)는 거의 이뤄지지 않아, 수요과 공급의 법칙으로 보면 공급은 많은데 수요가 많지않아 집값 하락이 이루져졌다고 예상된다. 
    - 세종은 극단적으로 4개년 동안 미분양된 주택이 없다고 볼 수 있을 정도로 집이 부족하였고, 그에 비해 인구 유입이 둔화되고 있지만 그래도 많기 때문에, 수요는 증가하는데 공급이 극단적으로 적어 시군구 중에 가장 높은 집값 상승률을 보여줬다고 예상한다.

In [ ]:
### 미분양 주택 전처리
def unsold_home_prepared(unsold_home):
    col1 = ["date"]
    col2 = [i[0:-3] for i in unsold_home.columns[1:].tolist()]
    col = col1 + col2
    unsold_home.columns = col
    unsold_home = unsold_home.fillna(0)
    unsold_home["date"] = pd.to_datetime(unsold_home["date"], format="%Y-%m-%d")
    unsold_home["year"] = [i.year for i in unsold_home["date"]]
    
    #년도 별 합산
    unsold_home_year = []
    for i in unsold_home.groupby("year"):
        subset = pd.DataFrame(i[1].sum(axis=0)).T
        subset["year"] = i[0]
        unsold_home_year.append(subset)
        
    unsold_home_year = pd.concat(unsold_home_year)
    return unsold_home_year

# unsold_home 전처리
unsold_home = unsold_home_prepared(unsold_home)

In [ ]:
unsold_home_17_20 = unsold_home.loc[unsold_home["year"].isin([2017,2018,2019, 2020])]

unsold_home_each = px.line(unsold_home_17_20, x="year", y=["경남", "세종"], markers=True)
unsold_home_each.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [i for i in range(2017, 2021)],
        ticktext = [f"{i}년" for i in range(2017, 2021)]),
    yaxis = dict(
        tickmode = 'array',
        tickvals = [i for i in range(0, 251000, 10000)],
        ticktext = [f"{i}세대" for i in range(0, 251000, 10000)]),
    margin=dict(l=0, r=0, t=0, b=0),
    template="plotly_dark",
    width = 600
)

![5](./images/5.png)

In [ ]:
population_38 = population.loc[population["adm_nm"] == "경상남도"]
population_29 = population.loc[population["adm_nm"] == "세종특별자치시"]

### 전별도 대비 인구 증감율 구하기
pp_increase_38 = [population_38["house_cnt"].tolist()[i+1]/population_38["house_cnt"].tolist()[i]-1 for i in range(4)]
pp_increase_29 = [population_29["house_cnt"].tolist()[i+1]/population_29["house_cnt"].tolist()[i]-1 for i in range(4)]

population_38 = population_38.loc[population_38["year"].isin([2017,2018,2019,2020])]
population_38["ratio"] = pp_increase_38
population_38 = population_38.reset_index(drop=True)

population_29 = population_29.loc[population_29["year"].isin([2017,2018,2019,2020])]
population_29["ratio"] = pp_increase_29
population_29 = population_29.reset_index(drop=True)

population_29_38 = pd.concat([population_38, population_29])

population_29_38["adm_nm"] = ["경남", "경남", "경남", "경남", "세종", "세종", "세종", "세종"]

In [ ]:
population_29_38_fig = px.line(population_29_38, x="year", y="ratio", color="adm_nm")
population_29_38_fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [i for i in range(2017, 2021)],
        ticktext = [f"{i}년" for i in range(2017, 2021)]),
    yaxis = dict(
         tickmode = 'array',
         tickvals = [0, 0.05, 0.1, 0.15, 0.20],
         ticktext = ["0%", "5%", "10%", "15%", "20%"]),
    margin=dict(l=0, r=0, t=0, b=0),
    template="plotly_dark",
    width = 600
)

![6](./images/6.png)

---

### 4. 결론 <a class="anchor" id="chapter4"></a>

#### 4-1. 한계점 <a class="anchor" id="section_4_1"></a>
- 통계적 기법의 부재
    - 여러 데이터의 교집합된 데이터가 얼마 없다고 판단하여.... 통계적인 기법을 사용하기에는 표본이 부족하다고 판단하여 단순한 추론에서 마무리 지었다. 
- 집값에 대한 다양한 세부 요인 존재하기 때문에 단순하게 결론을 지으면 안된다고 생각한다. 그렇기에 데이터를 보고 단순한 추론에서 마루리 하였다.
    - 물가 상승 
    - 원자재 값 상승 
    - 인건비 상승
- 극단적인 타겟 설정으로 인해 주제에 부합하게 만든 느낌이 있기 때문에 추후 분석을 더 진행하게 된다면 더 다양한 지역을 같이 분석하는게 좋을 것 같다. 

#### 4-2. 추후 연구 방향 <a class="anchor" id="section_4_2"></a>

- 세부적인 요인을 추가하여 분석을 진행하면 좋을 것 같다. 
- 해당 데이터로 집값의 양극화를 분석해도 좋을 것 같다. 